In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [2]:
#x_train = pd.read_pickle('x_train.pkl')
x_test = scaler.fit_transform(pd.read_pickle("x_test.pkl").iloc[:,12:])
y_train = scaler.fit_transform(pd.read_pickle('y_train.pkl'))
y_test = scaler.fit_transform(pd.read_pickle("y_test.pkl"))

In [3]:
#x_test = x_test.iloc[:,12:]
x_test

array([[0.01191766, 0.00975081, 0.00325027, ..., 0.00713436, 0.00237812,
        0.00713436],
       [0.00216685, 0.01191766, 0.00975081, ..., 0.01189061, 0.00713436,
        0.00237812],
       [0.00325027, 0.00216685, 0.01191766, ..., 0.0059453 , 0.01189061,
        0.00713436],
       ...,
       [0.00216685, 0.        , 0.00433369, ..., 0.        , 0.00237812,
        0.00475624],
       [0.00216685, 0.00216685, 0.        , ..., 0.0059453 , 0.        ,
        0.00237812],
       [0.00541712, 0.00216685, 0.00216685, ..., 0.00356718, 0.0059453 ,
        0.        ]])

In [4]:
np.shape(np.array(x_test))

(1195208, 61)

In [5]:
forecast_horizon = 1
es_base = []

In [6]:
# train the ES model:
import warnings
warnings.filterwarnings("ignore", category=Warning)

In [8]:

for i in tqdm(range(len(x_test))):
    ES = SimpleExpSmoothing(x_test[i,:])
    result = ES.fit()
    forc = result.forecast(steps=forecast_horizon)
    es_base.append(forc)
    #tru = y_test[i]
    #es_mase.append(forecast_mase(tru,forc,1))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1195208/1195208 [1:29:22<00:00, 222.87it/s]


In [9]:
np.shape(es_base)

(1195208, 1)

In [11]:
es_b = np.array(es_base).reshape(3049,392)
tru = np.array(y_test).reshape(3049,392)

In [12]:
tru

array([[0.00216685, 0.00325027, 0.00216685, ..., 0.        , 0.        ,
        0.        ],
       [0.00216685, 0.00108342, 0.        , ..., 0.        , 0.        ,
        0.00108342],
       [0.00108342, 0.00216685, 0.00216685, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.01191766, 0.00975081, 0.00433369, ..., 0.00108342, 0.        ,
        0.00216685],
       [0.01408451, 0.01408451, 0.01733478, ..., 0.00108342, 0.00108342,
        0.        ],
       [0.04550379, 0.02816901, 0.01408451, ..., 0.00216685, 0.00541712,
        0.00108342]])

In [13]:
def forecast_mase(truth_1,forecast_1, epsilon):
    from sklearn.metrics import mean_absolute_error
    mase = []
    forecast = np.array(forecast_1)
    truth = np.array(truth_1)
    tru_diff = np.abs(truth[:, 1:] - truth[:, :-1])  # achieve .diff()
    mean_diff = np.mean(tru_diff, axis=1)
    for i in range(len(forecast)):
        mae_ = mean_absolute_error( truth[i, :],forecast[i, :])
        mean_diff_i = mean_diff[i] if mean_diff[i] != 0 else epsilon
        mase.append(mae_ / mean_diff_i)
    return mase

In [13]:
'''
##### base forecasts:
re_base_all = []
re_base_216 = []
for i in range(len(es_b)):
    pred = es_b[i].reshape(14,28)
    truth = tru[i].reshape(14,28)
    re_base_all.append(forecast_mase(truth, pred, 1))
    re_base_216.append(np.mean(re_base_all))'''

'\n##### base forecasts:\nre_base_all = []\nre_base_216 = []\nfor i in range(len(es_b)):\n    pred = es_b[i].reshape(14,28)\n    truth = tru[i].reshape(14,28)\n    re_base_all.append(forecast_mase(truth, pred, 1))\n    re_base_216.append(np.mean(re_base_all))'

In [14]:
'''
np.mean(re_base_216)'''

'\nnp.mean(re_base_216)'

In [14]:
#### Summing Matrix
l1 = np.ones(10).reshape(1,10)
l2 = [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]]
l3 = np.eye(10)
S_Matrix = np.concatenate((l1, l2, l3), axis=0)
################# TD method:
# G matrix for TD, proportions of historical averages
def G_Matrix_TD(df ,ts_length, ts_niah, ts_nibl):
    n = int(len(df)/(ts_length*ts_niah))
    G_td = []
    df_ = np.array(df).reshape(n,int(len(df)/n))
    for i in tqdm(range(n)):
        ts = df_[i].reshape(ts_niah,ts_length)
        prop = (np.mean(ts[-10:,:],axis = 1)/np.mean(ts[0,:])).reshape(ts_nibl,1)
        zero = np.zeros((ts_nibl,ts_niah-1))
        g_td = np.concatenate((prop, zero), axis=1)
        G_td.append(g_td)
    return G_td
# Different G matrix for 3 methods:
################# BU method:
# G matrix for BU
g1 = np.zeros((10,4))#（ bottom_level , except_bottom level）
g2 = np.eye(10)# bottom level
G_bu = np.concatenate((g1,g2), axis=1)

################# TD method:
# parameters: bottom level data, top level data, time series length, 
G_td = G_Matrix_TD(y_train, 1852, 14, 10)

################# MinT method:
I = np.ones(10)
kh= 1
W = np.diag(S_Matrix@I)
G_mint = np.linalg.inv(S_Matrix.T@np.linalg.inv(W)@S_Matrix)@S_Matrix.T@np.linalg.inv(W)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 3049/3049 [00:00<00:00, 11901.79it/s]


In [15]:
################## Different SG matrix for 3 methods:
SG_bu = np.dot(S_Matrix, G_bu)
SG_mint = np.dot(S_Matrix,G_mint)
SG_td = []
for i in tqdm(range(len(G_td))):
    sg_td = S_Matrix@G_td[i]
    SG_td.append(sg_td)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 3049/3049 [00:00<00:00, 234499.55it/s]


In [16]:
##### base forecasts:
re_base_all = []
re_base_216 = []
for i in range(len(es_b)):
    pred = es_b[i].reshape(14,28)
    truth = tru[i].reshape(14,28)
    re_base_all.append(forecast_mase(truth, pred, 1))
    re_base_216.append(np.mean(re_base_all))
##### bu forecasts:
re_bu_all = []
re_bu_216 = []
for i in range(len(es_b)):
    pred = es_b[i].reshape(14,28)
    truth = tru[i].reshape(14,28)
    y_coh_frec_bu = np.dot(SG_bu, pred)
    re_bu_all.append(forecast_mase(truth, y_coh_frec_bu, 1))
    re_bu_216.append(np.mean(re_bu_all))
##### td forecasts:
re_td_all = []
re_td_216 = []
for i in range(len(es_b)):
    pred = es_b[i].reshape(14,28)
    truth = tru[i].reshape(14,28)
    y_coh_frec_td = np.dot(SG_td[i], pred)
    re_td_all.append(forecast_mase(truth,y_coh_frec_td, 1))
    re_td_216.append(np.mean(re_td_all))
##### mint forecasts:
re_mint_all = []
re_mint_216 = []
for i in range(len(es_b)):
    pred = es_b[i].reshape(14,28)
    truth = tru[i].reshape(14,28)
    y_coh_frec_mint = np.dot(SG_mint, pred)
    re_mint_all.append(forecast_mase(truth, y_coh_frec_mint, 1))
    re_mint_216.append(np.mean(re_mint_all))

In [17]:
print("ES's forecasts:", np.mean(re_base_216))
print("ES with BU's forecasts:", np.mean(re_bu_216))
print("ES with TD's forecasts:", np.mean(re_td_216))
print("ES with MinT's forecasts:", np.mean(re_mint_216))

ES's forecasts: 1.011334343082247
ES with BU's forecasts: 1.0071222573139647
ES with TD's forecasts: 1.0374173386780852
ES with MinT's forecasts: 1.0118783479316238


In [18]:
# np.shape(results) : 
def LevelMase(results, level_sta,level_gap,):
    level_mase = []
    if level_sta == 0:
        for i in range(len(results)):
            level_mase.append(results[i][level_sta])
    elif level_sta != 0:
        for i in range(len(results)):
            level_mase.append(results[i][level_sta:level_sta+level_gap])        
    return level_mase

In [23]:
l1_base_216 = LevelMase(re_base_all,0,0)
l2_base_216 = LevelMase(re_base_all,1,3)
l3_base_216 = LevelMase(re_base_all,4,10)
print("ES's base forecasts' L1 MASE:", np.mean(l1_base_216))
print("ES's base forecasts' L2 MASE:", np.mean(l2_base_216))
print("ES's base forecasts' L3 MASE:", np.mean(l3_base_216))

ES's base forecasts' L1 MASE: 2.5855044835584815
ES's base forecasts' L2 MASE: 1.1568612557899505
ES's base forecasts' L3 MASE: 1.0199019711821045


In [20]:
l1_bu_216 = LevelMase(re_bu_all,0,0)
l2_bu_216 = LevelMase(re_bu_all,1,3)
l3_bu_216 = LevelMase(re_bu_all,4,10)
print("ES's BU forecasts' L1 MASE:", np.mean(l1_bu_216))
print("ES's BU forecasts' L2 MASE:", np.mean(l2_bu_216))
print("ES's BU forecasts' L3 MASE:", np.mean(l3_bu_216))

ES's BU forecasts' L1 MASE: 2.4797570629797625
ES's BU forecasts' L2 MASE: 1.141846797245144
ES's BU forecasts' L3 MASE: 1.0199019711821045


In [21]:
l1_td_216 = LevelMase(re_td_all,0,0)
l2_td_216 = LevelMase(re_td_all,1,3)
l3_td_216 = LevelMase(re_td_all,4,10)
print("ES's TD forecasts' L1 MASE:", np.mean(l1_td_216))
print("ES's TD forecasts' L2 MASE:", np.mean(l2_td_216))
print("ES's TD forecasts' L3 MASE:", np.mean(l3_td_216))

ES's TD forecasts' L1 MASE: 2.5855044835584815
ES's TD forecasts' L2 MASE: 1.1905276560174918
ES's TD forecasts' L3 MASE: 1.064919833797746


In [22]:
l1_mint_216 = LevelMase(re_mint_all,0,0)
l2_mint_216 = LevelMase(re_mint_all,1,3)
l3_mint_216 = LevelMase(re_mint_all,4,10)
print("ES's MinT forecasts' L1 MASE:", np.mean(l1_mint_216))
print("ES's MinT forecasts' L2 MASE:", np.mean(l2_mint_216))
print("ES's MinT forecasts' L3 MASE:", np.mean(l3_mint_216))

ES's MinT forecasts' L1 MASE: 2.5322615369928028
ES's MinT forecasts' L2 MASE: 1.1516857106675709
ES's MinT forecasts' L3 MASE: 1.0256745810577583


In [29]:
np.max(re_base_all)

1063.7338746626947